# Working with BIDS data

In [ ]:
import matplotlib
import pathlib
import os

import mne
import mne_bids

matplotlib.use('Qt5Agg')

## Read the input data we wish to convert to BIDS

In [ ]:
sample_data_folder = '/Users/christinadelta/datasets/eeg_testing_data'
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'data',
                                    'sample_audvis_raw.fif')

# raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
raw = mne.io.read_raw_fif(sample_data_raw_file)

events = mne.find_events(raw)
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}

## Write the raw data to BIDS

We need to specify the power line frequency.

In [ ]:
raw.info['line_freq'] = 60

## Adding subject info to the raw.info structure 

If participant information is not given with the raw, we can add this info in the raw.info structure before writing data to BIDS. We just need to create a py dictionary called ```subject_info```, with a few keys (e.g. id, sex, hand, birthday)

In [ ]:
subject_info = {
    'birthday': (1988, 10, 1),
    'sex': 2,
    'hand': 3   
}

# now we should assign the subject_info to the raw.info structure
raw.info['subject_info'] = subject_info
raw.info

Now, actually write the data.

In [ ]:
out_path = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True)

## Write MEGIN / Elekta / NeuroMag fine-calibration and crosstalk files

In [ ]:
# first make the directory writable for the mne-bids stuff
sample_data_dir = pathlib.Path(sample_data_folder)
cal_path = sample_data_dir / 'SSS' / 'sss_cal_mgh.dat'
ct_path = sample_data_dir / 'SSS' / 'ct_sparse_mgh.fif'

mne_bids.write_meg_calibration(cal_path, bids_path=bids_path)
mne_bids.write_meg_crosstalk(ct_path, bids_path=bids_path)


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Write the same raw data again, but this time act as if it were from a different subject – subject "02".</li>
    </ul>
</div>

In [ ]:
# write mne-bids for subejct 2
bids_path = mne_bids.BIDSPath(subject='02',
                              session='01',
                              task='audiovisual',
                              run='01',
                              root=out_path)

mne_bids.write_raw_bids(raw, bids_path=bids_path, events_data=events,
                        event_id=event_id, overwrite=True)

## Print the created file & directory structure

In [ ]:
mne_bids.print_dir_tree(out_path)

## Generate a data summary

In [ ]:
print(mne_bids.make_report(out_path))

## Reading BIDS data

Let's try to read back the data we just stored according to the BIDS specification.

In [ ]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)

In [ ]:
raw.plot()

## Events are stored as annotations – but we convert between the two.

Let's look at some of the first annotation simply by specifying their index

In [ ]:
raw.annotations[0]
# raw.annotations[1]
# raw.annotations[2]

In [ ]:
events, event_id = mne.events_from_annotations(raw)

In [ ]:
mne.viz.plot_events(events, event_id=event_id, sfreq=raw.info['sfreq'])

## Fine the MEG fine-calibration and crosstalk files

In [ ]:
bids_path.meg_calibration_fpath

In [ ]:
bids_path.meg_crosstalk_fpath